In [1]:
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from torchvision.transforms import functional as F
import cv2
import numpy as np

# Define a filter for target objects
TARGET_OBJECTS = ["bottle", "cup"]  # Adjust as needed for soda cans, water bottles, etc.

/home/riadoshi/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-03 04:13:24.140213: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 04:13:24.186427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 04:13:24.186461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 04:

In [2]:
# Load pre-trained DETR model and image processor
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Define a filter for target objects
TARGET_OBJECTS = ["bottle", "cup", "orange"]  # Replace with soda cans, water bottles, etc.

def detect_objects(image: np.ndarray):

    # Convert the image to RGB and preprocess it
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    inputs = processor(images=rgb_image, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract logits, boxes, and probabilities
    logits = outputs.logits[0]  # Class logits
    boxes = outputs.pred_boxes[0]  # Bounding boxes (normalized [0, 1])

    # Compute class probabilities
    probs = logits.softmax(-1)
    confidences, class_ids = probs[..., :-1].max(-1)  # Ignore "no-object" class

    # Denormalize bounding boxes
    img_height, img_width = rgb_image.shape[:2]
    boxes = boxes * torch.tensor([img_width, img_height, img_width, img_height], dtype=torch.float)

    # Filter results
    id2label = model.config.id2label
    results = []
    for i, confidence in enumerate(confidences):
        if confidence > 0.5:  # Adjust confidence threshold if needed
            label = id2label[class_ids[i].item()]
            if label in TARGET_OBJECTS:
                x_center, y_center, width, height = boxes[i].tolist()
                x1 = int(x_center - width / 2)
                y1 = int(y_center - height / 2)
                x2 = int(x_center + width / 2)
                y2 = int(y_center + height / 2)
                results.append({
                    "label": label,
                    "bounding_box": [x1, y1, x2, y2],
                    "confidence": float(confidence)
                })
    return results

In [9]:
# Load a sample image as a numpy array given a path
image_path = '/home/riadoshi/106a-final/test_images/spray_NR.png'
image = cv2.imread(image_path)

# Perform detection
results = detect_objects(image)

# Print results
for result in results:
    print(f"Detected {result['label']} with confidence {result['confidence']:.2f} at {result['bounding_box']}")

Detected bottle with confidence 0.97 at [9, 47, 540, 1482]
